# **Modelling and Evaluation Notebook**

## Objectives
Answer Business requirement 2: Binary Classification using Convolutional Neural Networks

* predict if a given leaf is infected or not judging by the presence of powdery mildew.
* use the CNN to map relationships between features and labels.
* build a binary classifier and generate reports.

## Inputs

* inputs/cherry-leaves-dataset/cherry-leaves/train
* inputs/cherry-leaves-dataset/cherry-leaves/test
* inputs/cherry-leaves-dataset/cherry-leaves/validation
* image shape embeddings pickle file

## Outputs
TODO
* v1:
    * class_distribution.png  
    * class_indices.pkl  
    * evaluation.pkl
    * model_v1.keras  
    * training_accuracy_v1.png  
    * training_loss_v1.png 

## Additional Comments
TODO
* In case you have any additional comments that don't fit in the previous bullets, please state them here. 



---

### Import regular packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

### Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

### Set input directory

In [4]:
my_data_dir = 'inputs/cherry-leaves-dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

### Set output directory

In [ ]:
version = 'v2'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('Old version is already available, create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Gather labels

In [ ]:
labels = os.listdir(train_path)

print(f"Project Labels: {labels}")

### Load image shape embeddings

In [ ]:
## Import saved image shape embedding pickle file
import joblib
version = 'v1'  # from v1 original
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

***

# Review class distribution

* across whole dataset
* per train, test, and validation

In [ ]:
df_freq = pd.DataFrame([])
total_images_count = 0

# gather info
for folder in ['train', 'validation', 'test']:
    for label in labels:

        path = my_data_dir + '/' + folder + '/' + label
        
        image_count = int(len(os.listdir(path)))
        total_images_count += image_count

        df_freq = df_freq.append(pd.Series(
            {'Set': folder,
             'Label': label,
             'Frequency': image_count},
            ), ignore_index=True )
        print(f"* {folder}- {label}: {image_count} images\n")

print(f'{total_images_count} images total')
print('--------')

### plot class distribution
plt.figure(figsize=(10, 6))
sns.barplot(x='Set', y='Frequency', hue='Label', data=df_freq)
plt.title('Class Distribution')
plt.savefig(f'{file_path}/class_distribution.png', bbox_inches='tight', dpi=600)
plt.show()
print('\n')

print('--------')

# confirm percentages of dataset
df_freq.set_index('Label', inplace=True)
df_freq['Percent of DataSet'] = round(df_freq['Frequency'] / total_images_count * 100)
print(df_freq)




We can confirm that train, validation and test set percentages of dataset are split as expected, and that there are equal amounts of both classes (healthy and powdery_mildew) in each set.

***

# Image Augmentation

### Define image data generator, initialize


In [ ]:
# This function generates batches of image data with real-time data augmentation.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize
augmented_image_data = ImageDataGenerator(rotation_range=30,  # increase from 20
                                          width_shift_range=0.15,  # increase from 0.1
                                          height_shift_range=0.15,  # increase from 0.1
                                          brightness_range=[0.8, 1.2],  # added
                                          shear_range=0.1,
                                          zoom_range=0.2,  # increase from 0.1
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

### Augment TRAINING image dataset


In [ ]:
batch_size = 16  # try small batch size for small dataset v2
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True,
                                                     seed=42  # add seed for consistency
                                                     )

train_set.class_indices

### Augment VALIDATION image dataset


In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

### Augment TEST image dataset

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

### Plot augmented training images

In [ ]:
for _ in range(20):
    img, label = train_set.next()
    print(f'{img.shape}\n')  # expect: (20, 256, 256, 3)
    plt.imshow(img[0])
    print('--------------')
    plt.show()
    

### Plot augmented validation and test images

In [ ]:
for _ in range(10):  
    img, label = validation_set.next()
    print(f'{img.shape}\n')
    plt.imshow(img[0])
    print('--------------')
    plt.show()


###  Observations
Augmented validation and test images have been standardized between 0 to 255 pixels. As you can see, the images are ugmented and are ready to be used for developing and training a CNN model.

### Save class indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

---

# Model Creation

---

### ML Model

* Import model packages

In [18]:
import tensorflow as tf
import keras_tuner as kt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization



RuntimeError: module compiled against API version 0xf but this version of numpy is 0xd

ImportError: initialization failed

* ### Model

In [21]:
def create_tf_model():
    """
    Creates a CNN model for binary 
    classification of leaf images
    """
    model = Sequential()

    # Input layer: CONV1
    model.add(Conv2D(filters=32, kernel_size=(3, 3),  # find dominant pixels
        input_shape=image_shape,  # average image shape
        activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # CONV2
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
        activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # CONV3
    model.add(Conv2D(
        filters=128,  # increase
        kernel_size=(3,3),
        activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Flatten
    model.add(Flatten())

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(Dense(128, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile
    model.compile(
        loss='binary_crossentropy',
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics=[
            'accuracy',
        ])

    return model

## Instantiate tuner and perform hypertuning

In [ ]:
# TODO evaluate https://www.tensorflow.org/tutorials/keras/keras_tuner

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=my_data_dir
                     )

In [ ]:
summary = create_tf_model().summary()

* Early Stopping

    * Avoid overfitting

In [54]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=3)  # increase patience from 3 to 10

### Run hypertune search

In [ ]:
# SOURCE https://www.tensorflow.org/tutorials/keras/keras_tuner

# tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# # Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)
# _________
# # then reinstantiate hyermodel and train it with optimal number of epochs from above
# hypermodel = tuner.hypermodel.build(best_hps)

# # Retrain the model
# hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)
# _________

# eval_result = hypermodel.evaluate(img_test, label_test)
# print("[test loss, test accuracy]:", eval_result)


### Visaulise Model

In [ ]:
# TODO remove if neccessary
from tensorflow.keras.utils import plot_model

model = create_tf_model

plot_model(model, show_shapes=True)

***

# Create Model

In [ ]:
# # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
# model = tuner.hypermodel.build(best_hps)
# history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

# val_acc_per_epoch = history.history['val_accuracy']
# best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
# print('Best epoch: %d' % (best_epoch,))

model = create_tf_model()
model.fit(train_set,
          epochs=25,
          steps_per_epoch=len(train_set.classes) // batch_size,
          validation_data=validation_set,
          callbacks=[early_stop],
          verbose=1
          )

### Save model

In [ ]:
model.save(f'outputs/{version}/cherry-tree-model.h5')

*** 

# Evaluate Model Performance 

In [72]:
def confusion_matrix_and_report(X,y,pipeline,label_map):
    prediction = pipeline.predict(X).reshape(-1)
    prediction = np.where(prediction<0.5,0,1)
    # the prediction using sigmoid as an activation function is a probability number, between 0 and 1
    # we convert it to 0 or 1, if it is lower than 0.5, the predicted class is 0, otherwise it is 1
    # you could change the threshold if you want.
    print('--- Confusion Matrix ---')
    print(
        pd.DataFrame(confusion_matrix(y_true=prediction, y_pred=y),
        columns=[ ["Actual " + sub for sub in label_map] ],
        index= [ ["Prediction " + sub for sub in label_map ]]
        ))
    print("\n")
    print('--- Classification Report ---')
    print(classification_report(y, prediction, target_names=label_map),"\n")


def clf_performance(X_train,y_train,X_test,y_test,X_val, y_val,pipeline,label_map):
    print("#### Train Set #### \n")
    confusion_matrix_and_report(X_train,y_train,pipeline,label_map)

    print("#### Validation Set #### \n")
    confusion_matrix_and_report(X_val,y_val,pipeline,label_map)

    print("#### Test Set ####\n")
    confusion_matrix_and_report(X_test,y_test,pipeline,label_map)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss', 'val_loss']].plot(style='.-')
plt.title('Loss')
plt.savefig(f'{file_path}/training_loss.png', bbox_inches='tight', dpi=600)
print('\n')
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title('Accuracy')
plt.savefig(f'{file_path}/training_accuracy.png', bbox_inches='tight', dpi=600)

### Evaluate and save

In [ ]:
evaluation = model.evaluate(test_set)

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v1/evaluation.pkl")

# Run Live Prediction 

from tensorflow.keras.preprocessing import image

pointer = 66  # TODO change to random within length of dir
label = labels[1]  # select Uninfected or Parasitised # TODO change to random (0 or 1) and print image class too

pil_image = image.load_img(test_path + '/' + label + '/' + os.listdir(test_path+'/' + label)[pointer],
                           target_size=image_shape, color_mode='rgb')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
pil_image

In [ ]:
pred_img = image.img_to_array(pil_image)
pred_img = np.expand_dims(pred_img, axis=0)/255
print(pred_img.shape)
pred_img

In [ ]:
# predict class probability on test image
pred_proba = model.predict(pred_img)[0, 0] # why 0 0?

target_map = {v: k for k, v in train_set.class_indices.items()}
pred_class = target_map[pred_proba > 0.5]

if pred_class == target_map[0]:
  pred_proba = 1 - pred_proba

print(f'Prediction: {pred_class}\nConfidence: {pred_proba*100:.1f}%') # TODO edit syntax/.1f to more?

## Evaluation:

Given that the training data, whilst augmented was very small, I believe this has caused overfitting and a lack of proper deep learning, leading to 100% certainty.
Now that I have defined a first iteration of model 1, 

## Next steps:
* Define clear pipelines 
    * clear normalisation
    * clear image augmentation
        * more img aug parameters including:
            * brightness
            * color
            * random cropping
            * rotation
    * clear hypertuning 
    * clear evaluation process: 
        * plot confusion matrix
        * classicication report

*** 

In [ ]:
import joblib
evaluation = joblib.load('outputs/v1/evaluation.pkl')
evaluation # loss, accuracy? TODO confirm

# Model v2